In [1]:
import os
import cv2
import torch
from IPython.display import Image
from random import choice
import shutil
import glob
import numpy as np
import shutil
import re
from collections import defaultdict
from PIL import Image
import subprocess


###path
all_video_input = "/Users/yayhaeslami/Desktop/my_workspace/sign_language/preparing_dataset/all_video"
one_video_input = "/Users/yayhaeslami/Desktop/my_workspace/sign_language/preparing_dataset/all_video/bebakhshid1.mp4"
video_in_video_folder = "/Users/yayhaeslami/Desktop/my_workspace/sign_language/preparing_dataset/1_video/video.mp4"

video_folder ="/Users/yayhaeslami/Desktop/my_workspace/sign_language/preparing_dataset/1_video"
convert_video_to_images_folder = "/Users/yayhaeslami/Desktop/my_workspace/sign_language/preparing_dataset/2_convert_video_to_images"
square_images_folder = "/Users/yayhaeslami/Desktop/my_workspace/sign_language/preparing_dataset/3_square_images"
resized_images_folder = "/Users/yayhaeslami/Desktop/my_workspace/sign_language/preparing_dataset/4_resized_images"
dataset_end = "/Users/yayhaeslami/Desktop/my_workspace/sign_language/preparing_dataset/dataset"

yolo_folder = '/Users/yayhaeslami/Desktop/my_workspace/sign_language/preparing_dataset/yolov5'
detect_folder = "/Users/yayhaeslami/Desktop/my_workspace/sign_language/preparing_dataset/yolov5/runs/detect"
labels_folder = "/Users/yayhaeslami/Desktop/my_workspace/sign_language/preparing_dataset/yolov5/runs/detect/exp/labels"
yolo_weights_path = "/Users/yayhaeslami/Desktop/my_workspace/sign_language/preparing_dataset/yolov5/runs/train/weights/best.pt"

#name class
name_class = "salam"





# مسیرهای مورد نظر
paths = [
    video_folder,
    convert_video_to_images_folder,
    square_images_folder,
    resized_images_folder,
    yolo_weights_path,
    dataset_end
]

# ایجاد فولدرها
for path in paths:
    try:
        os.makedirs(path)
        print(f"Folder created: {path}")
    except FileExistsError:
        print(f"Folder already exists: {path}")
 



### 1_ Deleted all files



def delete_files_and_folders(pathsـdelete_files):
    for folder_path in pathsـdelete_files:
        items = glob.glob(os.path.join(folder_path, "*"))

        for item in items:
            try:
                if os.path.isfile(item) or os.path.islink(item):
                    os.remove(item)  
            
                elif os.path.isdir(item):
                    shutil.rmtree(item)  
                    print(f"Deleted folder: {item}")
            except Exception as e:
                print(f"Error deleting {item}: {e}")


### 2_ cut up and down video and change name video

def crop_video(video_path_change_name_video, save_path_change_name_video):

    video_capture = cv2.VideoCapture(video_path_change_name_video)

    width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

    min_size = min(width, height)
    start_x = (width - min_size) // 2
    start_y = (height - min_size) // 2

    crop_region = (start_x, start_y, min_size, min_size)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
    video_writer = cv2.VideoWriter(save_path_change_name_video, fourcc, 30, (min_size, min_size))

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        cropped_frame = frame[start_y:start_y+min_size, start_x:start_x+min_size]


        video_writer.write(cropped_frame)


    video_capture.release()
    video_writer.release()
    cv2.destroyAllWindows()





### 3_ ran yolo model

def run_yolov5_detection(video_source, weights_path, conf_threshold=0.4, vid_stride=1):
    """
    Runs YOLOv5 object detection on a video source.

    Parameters:
        video_source (str): Path to the video file or directory containing images.
        weights_path (str): Path to the YOLOv5 weights file.
        conf_threshold (float, optional): Confidence threshold for detection. Defaults to 0.4.
        vid_stride (int, optional): Frame stride for video processing. Defaults to 1.
    """
    
    # Ensure the directory exists
    if not os.path.exists(yolo_folder):
        raise FileNotFoundError(f"Directory {yolo_folder} does not exist")
    
    # Construct the command with absolute paths
    detect_script = os.path.join(yolo_folder, 'detect.py')
    command = [
        'python', detect_script,
        '--source', video_source,
        '--weights', weights_path,
        '--save-txt',
        '--vid-stride', str(vid_stride),
        '--conf-thres', str(conf_threshold)
    ]
    
    # Run the YOLOv5 detection script and capture output
    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        print("Command Output:", result.stdout)
        print("Command Error:", result.stderr)
    except subprocess.CalledProcessError as e:
        print("Error executing command:")
        print(e.output)
        print(e.stderr)
        raise



###4_ extract frames from video
def convert_video_to_images(video_path, output_folder):
    base_name = os.path.basename(video_path).split('.')[0]
    
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    count = 0
    frame_interval = 1 

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    while success:
        if count % frame_interval == 0:
            image_path = os.path.join(output_folder, f"{base_name}_{count // frame_interval + 1}.jpg")
            cv2.imwrite(image_path, image) 
        success, image = vidcap.read()
        count += 1

    print(f"{count // frame_interval} frames extracted.")




###5_ Convert photos to black and white

def convert_photos_to_grayscale(input_folder):
    """
    Converts photos in the input folder to grayscale and saves them in the output folder.

    Args:
        input_folder (str): The path to the folder containing the images.
        output_folder (str): The path to the folder where the grayscale images will be saved.
    """
    if not os.path.exists(input_folder):
        os.makedirs(input_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg"):
            input_path = os.path.join(input_folder, filename)
            
            img = cv2.imread(input_path)
            
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            

            output_path = os.path.join(input_folder, filename)

            cv2.imwrite(output_path, gray_img)
            
            print(f"Converted {filename} to grayscale and saved to {output_path}")

    print("Conversion completed.")


### 6_ add 1,2,3 to ferst file .txt with Right hand and left hand detection

def add_number_to_ferst_file(folder_path_labels):
    file_pattern = os.path.join(folder_path_labels , "*_1.txt")
    files = glob.glob(file_pattern)

    if files:
        file_path = files[0]
        with open(file_path, 'r+') as f:
            lines = f.readlines()
        
        hands = []
        others = []
        for line in lines:
            if line.startswith("1"):
                hands.append(line.strip())
            else:
                others.append(line.strip())
        
        hands_sorted = sorted(hands, key=lambda x: float(x.split()[1]))
        
        right_hand = ""
        left_hand = ""

        if len(hands_sorted) >= 1:
            right_hand = hands_sorted[0] + " 1"
        if len(hands_sorted) >= 2:
            left_hand = hands_sorted[1] + " 2"

        others = [coord + " 3" for coord in others]

        result = []
        if right_hand:
            result.append(right_hand)
        if left_hand:
            result.append(left_hand)
        result.extend(others)

        with open(file_path, 'w') as f:
            for line in result:
                f.write(line + "\n")

        print(f"file '{file_path}' Successfully updated.")
    else:
        print("No file with extension _1.txt was found in the desired path.")



######7_ Prepare text file for cut image

def Prepare_text_file_for_cut_image(directory_path_labels_folder):

    def extract_first_digit_after_decimal(line):
        parts = line.strip().split()
        first_digits_after_decimal = []
        for part in parts[1:5]:
            if '.' in part:
                fraction_part = part.split('.')[1]
                if len(fraction_part) > 0:
                    first_digits_after_decimal.append(int(fraction_part[0]))
                else:
                    first_digits_after_decimal.append(0)
            else:
                first_digits_after_decimal.append(0)
        return first_digits_after_decimal

    def extract_second_digit_after_decimal(line):
        parts = line.strip().split()
        second_digits_after_decimal = []
        for part in parts[1:5]:
            if '.' in part:
                fraction_part = part.split('.')[1]
                if len(fraction_part) > 1:
                    second_digits_after_decimal.append(int(fraction_part[1]))
                else:
                    second_digits_after_decimal.append(0)
            else:
                second_digits_after_decimal.append(0)
        return second_digits_after_decimal

    def count_matching_digits(list1, list2):
        return sum(1 for a, b in zip(list1, list2) if a == b)

    def read_file(file_path):
        with open(file_path, 'r') as file:
            lines = file.readlines()
        return [list(map(float, line.strip().split())) for line in lines]

    def write_file(file_path, lines):
        with open(file_path, 'w') as file:
            for line in lines:
                formatted_line = ' '.join([f"{int(num)}" if num.is_integer() else f"{num}" for num in line])
                file.write(formatted_line + '\n')

    def find_lines_with_sixth_element(lines, value):
        return [line for line in lines if len(line) > 5 and line[5] == value]

    def predict_next_coordinate(prev1, prev2):
        diff = np.array(prev2) - np.array(prev1)
        predicted = np.array(prev2) + diff
        return predicted

    def select_best_coordinate(predicted, candidates):
        best_candidate = None
        best_distance = float('inf')
        
        for candidate in candidates:
            candidate = np.array(candidate)
            distance = np.linalg.norm(candidate - predicted)
            
            if distance < best_distance:
                best_distance = distance
                best_candidate = candidate
        
        return best_candidate

    def process_files(file_path1, file_path2):
        with open(file_path1, 'r') as file1, open(file_path2, 'r') as file2:
            lines1 = file1.readlines()
            lines2 = file2.readlines()

        first_digits_list1 = [extract_first_digit_after_decimal(line) for line in lines1]
        second_digits_list1 = [extract_second_digit_after_decimal(line) for line in lines1]

        with open(file_path2, 'w') as file2:
            for line2 in lines2:
                first_digits2 = extract_first_digit_after_decimal(line2)
                second_digits2 = extract_second_digit_after_decimal(line2)
                matched_indices = []
                matched = False

                for i, first_digits1 in enumerate(first_digits_list1):
                    if first_digits1 == first_digits2:
                        matched_indices.append(i)

                if len(matched_indices) == 1:
                    parts = line2.strip().split()
                    matched_line_parts = lines1[matched_indices[0]].strip().split()
                    if len(matched_line_parts) > 5:
                        parts.append(matched_line_parts[5])
                    else:
                        parts.append('')  # Leave the 6th number empty
                    file2.write(' '.join(parts) + '\n')
                    matched = True
                elif len(matched_indices) > 1:
                    for i in matched_indices:
                        if second_digits_list1[i] == second_digits2:
                            parts = line2.strip().split()
                            matched_line_parts = lines1[i].strip().split()
                            if len(matched_line_parts) > 5:
                                parts.append(matched_line_parts[5])
                            else:
                                parts.append('')  # Leave the 6th number empty
                            file2.write(' '.join(parts) + '\n')
                            matched = True
                            break

                if not matched:
                    for i, first_digits1 in enumerate(first_digits_list1):
                        if count_matching_digits(first_digits1, first_digits2) >= 3:
                            parts = line2.strip().split()
                            matched_line_parts = lines1[i].strip().split()
                            if len(matched_line_parts) > 5:
                                parts.append(matched_line_parts[5])
                            else:
                                parts.append('')  # Leave the 6th number empty
                            file2.write(' '.join(parts) + '\n')
                            matched = True
                            break

                if not matched:
                    parts = line2.strip().split()
                    parts.append('')  # Leave the 6th number empty
                    file2.write(' '.join(parts) + '\n')
                    
        file1_lines = read_file(file_path1)
        file2_lines = read_file(file_path2)
        
        missing_values = [val for val in [1, 2, 3] if val not in [line[5] for line in file2_lines if len(line) > 5]]
        
        if missing_values:
            missing_value = missing_values[0]  
            
            
            file1_lines_with_missing = find_lines_with_sixth_element(file1_lines, missing_value)
            
            if file1_lines_with_missing:
                
                prev1 = file1_lines_with_missing[0][1:5]  
                prev2 = file1_lines_with_missing[1][1:5] if len(file1_lines_with_missing) > 1 else prev1 
                
            
                candidates = [line for line in file2_lines if len(line) <= 5 or (line[5] != 1 and line[5] != 2 and line[5] != 3)]
                candidate_coords = [line[1:5] for line in candidates]
                
                if candidate_coords:
                    
                    predicted = predict_next_coordinate(prev1, prev2)
                    
                    
                    best_coordinate = select_best_coordinate(predicted, candidate_coords)
                    
                    print(f"best{missing_value}:", best_coordinate)
                    
                    
                    for line in candidates:
                        if line[1:5] == best_coordinate.tolist():
                            line.append(float(missing_value))
                    
                    
                    write_file(file_path2, file2_lines)
                else:
                    print("There is no line without the sixth number or with the sixth number 1, 2, 3 in the second file.")
            else:
                print(f"No lines with numbers{missing_value} It is not present in the sixth position in the first file.")
        else:
            print("All numbers 1, 2 and 3 are in the sixth position of the second file.")

    def process_files_in_directory(directory):
        file_names = os.listdir(directory)
        file_names = [f for f in file_names if f.endswith('.txt')]
        file_names.sort(key=lambda x: int(x.split('_')[1].split('.')[0]))

        for i in range(len(file_names) - 1):
            file_path1 = os.path.join(directory, file_names[i])
            file_path2 = os.path.join(directory, file_names[i + 1])
            process_files(file_path1, file_path2)

    directory_path = directory_path_labels_folder
    process_files_in_directory(directory_path)



###8_ Deleted lines in files that haven't six number
def Deleted_lines_in_files_that_haven_not_six_number(directory_path_labels_folder):

    def read_file(file_path):
        with open(file_path, 'r') as file:
            lines = file.readlines()
        return lines

    def write_file(file_path, lines):
        with open(file_path, 'w') as file:
            for line in lines:
                file.write(line)

    def remove_lines_without_sixth_number(file_path):
        lines = read_file(file_path)
        filtered_lines = [line for line in lines if len(line.strip().split()) > 5]
        write_file(file_path, filtered_lines)

    def process_files_in_directory(directory):
        file_names = os.listdir(directory)
        file_names = [f for f in file_names if f.endswith('.txt')]

        for file_name in file_names:
            file_path = os.path.join(directory, file_name)
            remove_lines_without_sixth_number(file_path)

    directory_path = directory_path_labels_folder
    process_files_in_directory(directory_path)



###9_ Deleted file and image to 30
def Deleted_file_and_image(labels_directory, images_directory):
    label_files_before = set([f for f in os.listdir(labels_directory) if f.endswith('.txt')])

    def get_file_number(filename):
        return int(filename.split('_')[1].split('.')[0])

    label_files_before = sorted(label_files_before, key=get_file_number)

    def reduce_files_uniformly(files, target_count):
        total_files = len(files)
        if total_files <= target_count:
            return files
        
        step = total_files / (total_files - target_count)
        indices_to_remove = [int(i * step) for i in range(total_files - target_count)]
        
        for index in sorted(indices_to_remove, reverse=True):
            os.remove(os.path.join(labels_directory, files[index]))
        
        remaining_files = [f for f in os.listdir(labels_directory) if f.endswith('.txt')]
        remaining_files.sort(key=get_file_number)
        return remaining_files

    remaining_label_files = reduce_files_uniformly(label_files_before, 30)

    deleted_label_files = set(label_files_before) - set(remaining_label_files)

    for label_file in deleted_label_files:
        image_file = label_file.replace('.txt', '.jpg')
        image_path = os.path.join(images_directory, image_file)
        if os.path.exists(image_path):
            os.remove(image_path)

    remaining_image_files = [f for f in os.listdir(images_directory) if f.endswith('.jpg')]
    print(f"Number of text files remaining in labels: {len(remaining_label_files)}")
    print(f"Number of image files remaining in images: {len(remaining_image_files)}")

    def rename_files(directory, extension):
        files = [f for f in os.listdir(directory) if f.endswith(extension)]
        
        files.sort(key=lambda f: int(re.search(r'_(\d+)', f).group(1)))
        
        for i, filename in enumerate(files, start=1):
            new_filename = re.sub(r'_\d+', f'_{i}', filename)
            os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))

    rename_files(labels_directory, '.txt')

    rename_files(images_directory, '.jpg')
    rename_files(images_directory, '.png')
    rename_files(images_directory, '.jpeg')


###10_ cut image by text file

def cut_image_by_text_file(labels_folder_path, images_folder_path, save_folder):
    def convert_to_square(file_path):
        with open(file_path, 'r') as file:
            lines = file.readlines()
            modified_lines = []
            for line in lines:
                label, x_center, y_center, width, height, additional_number = map(float, line.split())
                
                max_size = max(width, height)
                
                x1 = max(x_center - max_size / 2, 0)
                y1 = max(y_center - max_size / 2, 0)
                x2 = x1 + abs(max_size)
                y2 = y1 + abs(max_size)
                
                modified_lines.append(f"{int(label)} {x1} {y1} {x2} {y2} {int(additional_number)}\n")
            
            return modified_lines

    def crop_and_save_squares(image_path, coordinates, save_folder):
        image = cv2.imread(image_path)
        image_name = os.path.splitext(os.path.basename(image_path))[0]

        for i, (label, (x1, y1), (x2, y2), additional_number) in enumerate(coordinates):
            x1_pixel = int(x1 * image.shape[1])
            y1_pixel = int(y1 * image.shape[0])
            x2_pixel = int(x2 * image.shape[1])
            y2_pixel = int(y2 * image.shape[0])

            cropped_image = image[y1_pixel:y2_pixel, x1_pixel:x2_pixel]

            cropped_save_path = os.path.join(save_folder, f"{image_name}_{additional_number}_{label}.jpg")
            cv2.imwrite(cropped_save_path, cropped_image)


    os.makedirs(save_folder, exist_ok=True)

    file_list = os.listdir(labels_folder_path)

    for file_name in file_list:
        if file_name.endswith(".txt"): 
            text_file_path = os.path.join(labels_folder_path, file_name)
            
            modified_lines = convert_to_square(text_file_path)
            
            coordinates = []
            for line in modified_lines:
                label, x1, y1, x2, y2, additional_number = map(float, line.split())
                coordinates.append((int(label), (x1, y1), (x2, y2), int(additional_number)))
            
            image_file_name = os.path.splitext(file_name)[0] + ".jpg"
            image_path = os.path.join(images_folder_path, image_file_name)
            if os.path.exists(image_path):

                crop_and_save_squares(image_path, coordinates, save_folder)
            else:
                print(f"{file_name} not found.")


###11_ change Dimensions to 128 * 128

def resize_images(input_folder_square_images_Dimensions, output_folder_resized_images_Dimensions, size):
    if not os.path.exists(output_folder_resized_images_Dimensions):
        os.makedirs(output_folder_resized_images_Dimensions)

    for filename in os.listdir(input_folder_square_images_Dimensions):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            img_path = os.path.join(input_folder_square_images_Dimensions, filename)
            image = cv2.imread(img_path)
            if image is not None:
                resized_image = cv2.resize(image, size)
                
                output_path = os.path.join(output_folder_resized_images_Dimensions, filename)
                
                cv2.imwrite(output_path, resized_image)
                print(f"Resized and saved: {filename}")
            else:
                print(f"Failed to read: {filename}")


###12_ mix 3 image to one with LDA

def mix_image_with_LDA(input_dir, output_base_dir, weights):
    image_groups = defaultdict(list)

    pattern = re.compile(r'^(.*?)_(\d+)_(\d+)')

    for filename in os.listdir(input_dir):
        match = pattern.match(filename)
        if match:
            #class_name = match.group(1)
            class_name = name_class
            group_id = match.group(2)
            weight_id = int(match.group(3))
            image_groups[class_name].append((os.path.join(input_dir, filename), group_id, weight_id))

    def load_image(image_path):
        with Image.open(image_path) as img:
            return np.array(img)

    def get_next_data_folder(base_dir):
        existing_folders = [d for d in os.listdir(base_dir) if d.startswith('data')]
        if not existing_folders:
            return os.path.join(base_dir, 'data1')
        existing_numbers = [int(re.search(r'data(\d+)', d).group(1)) for d in existing_folders if re.search(r'data(\d+)', d)]
        next_number = max(existing_numbers) + 1
        return os.path.join(base_dir, f'data{next_number}')


    for class_name, image_paths_groups_weights in image_groups.items():

        class_dir = os.path.join(output_base_dir, class_name)
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)


        data_dir = get_next_data_folder(class_dir)
        os.makedirs(data_dir, exist_ok=True)

        group_images = defaultdict(list)
        
        for image_path, group_id, weight_id in image_paths_groups_weights:

            image = load_image(image_path)


            weight = weights.get(weight_id, 1) 


            group_images[group_id].append((image, weight))

        for group_id, images_weights in group_images.items():

            combined_image = np.zeros_like(images_weights[0][0], dtype=np.float64)
            total_weight = 0

            for image, weight in images_weights:
                combined_image += image * weight
                total_weight += weight

            if total_weight > 0:
                combined_image /= total_weight

        
            output_path = os.path.join(data_dir, f"{group_id}.jpg")
            Image.fromarray(combined_image.astype(np.uint8)).save(output_path)

            print(f"image {output_path}  saved ")





import os

def ran_all(all_video_input):
    for video_path in os.listdir(all_video_input):
        video_in_folder = os.path.join(all_video_input, video_path)

        # 1: Delete files and folders
        paths_delete_files = [ 
            video_folder,
            convert_video_to_images_folder,
            square_images_folder,
            resized_images_folder,
            detect_folder
        ]

        delete_files_and_folders(paths_delete_files)

        # 2: Crop video
        video_path_change_name_video = video_in_folder
        save_path_change_name_video = video_in_video_folder

        crop_video(video_path_change_name_video, save_path_change_name_video)

        # 3: Run YOLOv5 detection
        run_yolov5_detection(
            video_source=video_in_video_folder,
            weights_path=yolo_weights_path,
            conf_threshold=0.4,
            vid_stride=1
        )

        # 4: Convert video to images
        video_path = video_in_video_folder
        output_folder = convert_video_to_images_folder

        convert_video_to_images(video_path, output_folder)

        # 5: Convert photos to grayscale
        input_folder = convert_video_to_images_folder

        convert_photos_to_grayscale(input_folder)

        # 6: Add number to the first file
        folder_path_labels = labels_folder

        add_number_to_ferst_file(folder_path_labels)

        # 7: Prepare text file for cutting images
        directory_path_labels_folder = labels_folder

        Prepare_text_file_for_cut_image(directory_path_labels_folder)

        # 8: Delete lines in files that haven't six numbers
        Deleted_lines_in_files_that_haven_not_six_number(directory_path_labels_folder)

        # 9: Delete file and image
        labels_directory = labels_folder
        images_directory = convert_video_to_images_folder

        Deleted_file_and_image(labels_directory, images_directory)

        # 10: Cut image by text file
        labels_folder_path = labels_folder
        images_folder_path = convert_video_to_images_folder
        save_folder = square_images_folder

        cut_image_by_text_file(labels_folder_path, images_folder_path, save_folder)

        # 11: Resize images
        input_folder_square_images_Dimensions = square_images_folder
        output_folder_resized_images_Dimensions = resized_images_folder
        size = (128, 128)

        resize_images(input_folder_square_images_Dimensions, output_folder_resized_images_Dimensions, size)

        # 12: Mix image with LDA
        input_dir = resized_images_folder
        output_base_dir = dataset_end
        weights = {
            1: 0.5,
            2: 0.5,  
            3: 0.5   
        }

        mix_image_with_LDA(input_dir, output_base_dir, weights)


########### ran 
ran_all(all_video_input)

Folder already exists: /Users/yayhaeslami/Desktop/my_workspace/sign_language/preparing_dataset/1_video
Folder already exists: /Users/yayhaeslami/Desktop/my_workspace/sign_language/preparing_dataset/2_convert_video_to_images
Folder already exists: /Users/yayhaeslami/Desktop/my_workspace/sign_language/preparing_dataset/3_square_images
Folder already exists: /Users/yayhaeslami/Desktop/my_workspace/sign_language/preparing_dataset/4_resized_images
Folder already exists: /Users/yayhaeslami/Desktop/my_workspace/sign_language/preparing_dataset/yolov5/runs/train/weights/best.pt
Folder already exists: /Users/yayhaeslami/Desktop/my_workspace/sign_language/preparing_dataset/dataset
Deleted folder: /Users/yayhaeslami/Desktop/my_workspace/sign_language/preparing_dataset/yolov5/runs/detect/exp
Command Output: 
Command Error: /Users/yayhaeslami/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' 

KeyboardInterrupt: 